In [ ]:
import numpy as np

import tensorflow as tf
print(tf.__version__)

import matplotlib.pyplot as plt

## Import Data

In [ ]:
data_name = f"Curve0407_Samp4K_Inc13_Freq5_4d32_4d74"
volt_name = f"DataSets/{data_name}_volt.txt"
ptrn_name = f"DataSets/{data_name}_pattern.txt"
inc_name = f"DataSets/{data_name}_inc.txt"
freq_name = f"DataSets/{data_name}_freq.txt"

In [ ]:
from emc_data_multi_engine import Data_Box

t_size = 0.2
data_box = Data_Box([volt_name,inc_name,freq_name,ptrn_name],
                    scales = [(0.0,18.0),(-30,30),(4.3,4.7),(0,0.01)])
data_box.export_scaler("scaler",["scale_0_18.save","scale_m30_30.save","scale_4d3_4d7.save","scale_0_0d01.save"])
x0_train,x0_test,x10_train,x10_test,x11_train, x11_test, y_train,y_test = data_box.gen_train_test(t_size = t_size)

x1_train = np.append(x10_train,x11_train,axis = -1)
x1_test = np.append(x10_test,x11_test,axis = -1)

_,_,dims = data_box.data_dim()

dim_volt,dim_inc,dim_freq,dim_ptrn = dims
dim_env = dim_inc+dim_freq

(n_train,n_test) = data_box.train_test_len()
_,inc_scaler,freq_scaler,prtn_scaler = data_box.scalers

## Build Models

### Initialize Engine

In [ ]:
from emc_pre_engine import EmC_Pre_Generator
from logger_engine import EmC_Logger
from emc_aux_fun import netsummary

### Build Network

In [ ]:
tf.keras.backend.clear_session()
emc_pre_generator = EmC_Pre_Generator(dim_volt,dim_ptrn,env_dim = dim_env)
emc_pre_generator.conv_paras(layer_num = 2,cluster = [3,2], channel = 1024, 
                             regu = 5e-8,env_on_all_conv = False)
emc_pre_generator.fc_paras(layer_num = 3,sz = 1600, regu = 5e-8)

(emc_pre,pred_ptrn_layer) = emc_pre_generator.create_EmC_Pre()
emc_pre.summary()

### Train

In [ ]:
foldername = "emc_pre_curve_incfqr"

logger = EmC_Logger(foldername)
(uid,dirname) = logger.get_ids() # Generate a random ID to track this network

logger.write_msg(f"Network hyperparameter summary: \n{emc_pre_generator}")
logger.write_msg(f"Network structure: \n{netsummary(emc_pre)}")
logger.write_msg(f"Training on dataset: {volt_name}, {ptrn_name}, test size = {t_size}")

In [ ]:
n_epochs = 50
learning_rate = 0.0003
batch_size = 50

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
lossfun = "mse"

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=dirname+"/network_para/epoch{epoch:03d}-loss{val_loss:.6f}.hdf5",
    monitor="val_loss",verbose=1,save_best_only=True,save_weights_only=True,mode="min")
emc_pre.compile(loss=lossfun, optimizer=opt)

logger.write_msg(f"# of epoch = {n_epochs} \n Batch size = {batch_size}\n Optimazation with Adam, learning rate {learning_rate}")

In [ ]:
logger.write_msg(f"Training starts")
history = emc_pre.fit([x0_train,x1_train], y_train, validation_data=([x0_test,x1_test], y_test), 
                      epochs=n_epochs, batch_size=batch_size, shuffle=True,
                      callbacks=[checkpoint])
logger.write_msg(f"Training finished")
np.savetxt(f"{dirname}/loss.txt",history.history['loss'])
np.savetxt(f"{dirname}/val_loss.txt",history.history['val_loss'])
logger.write_msg(f"Losses saved")

### Results

In [ ]:
num_epoch=list(range(1,n_epochs+1))
plt.figure(figsize=(6,4))

#Plot loss vs epoch
plt.rc('font', size=14) 
plt.plot(num_epoch, history.history['loss'])
plt.plot(num_epoch, history.history['val_loss'])
plt.legend(('Training Set','Validation Set'), fontsize=12, loc=1)
plt.xlabel('Epoch #', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.xlim(1,n_epochs)
plt.ylim(0,0.01)
plt.xticks(list(range(0,n_epochs,100)))

plt.show()

In [ ]:
y_predict = emc_pre.predict([x0_test,x1_test])
errs = emc_pre.evaluate([x0_test,x1_test],y_test)

theta = np.arange(0, np.pi+np.pi/36, np.pi/36)

plt.figure(figsize=(15,15))

for n_example in range (0,25):
    y_example = prtn_scaler.inverse_transform(y_test[n_example,:].reshape(1, -1))
    y_predictn = prtn_scaler.inverse_transform(y_predict[n_example,:].reshape(1, -1))
    a_example = inc_scaler.inverse_transform(x1_test[n_example,:].reshape(1,-1))[0,0]
    f_example = freq_scaler.inverse_transform(x1_test[n_example,:].reshape(1,-1))[0,1]
    
    #plt.subplot(5,5,n_example+1,projection = 'polar')
    plt.subplot(5,5,n_example+1)
    plt.rc('font', size=8)
    plt.plot(theta, y_predictn.reshape(dim_ptrn,1))
    plt.plot(theta, y_example.reshape(dim_ptrn,1))
    plt.text(1,0.006,f"{a_example:.0f} deg", fontsize=12)
    plt.text(1,0.005,f"{f_example:.2f} GHz", fontsize=12)
    plt.ylim(0,0.007)
    if n_example == 0: plt.legend(('Prediction','Real'), fontsize=12, loc=3)